In [12]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score,
    adjusted_rand_score, normalized_mutual_info_score
    )


En primer lugar vamos a cargar la base de datos

In [13]:
df= pd.read_csv('age_gender.csv')
df.isnull()

,age,ethnicity,gender,img_name,pixels
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2569,False,False,False,False,False
2570,False,False,False,False,False
2571,False,False,False,False,False
2572,False,False,False,False,False


In [26]:
df.describe()

,age,ethnicity,gender
count,2574.000000,2574.000000,2574.000000
mean,9.854701,1.357809,0.501943
std,12.917293,1.452374,0.500093
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,11.000000,1.000000,1.000000
75%,16.000000,3.000000,1.000000
max,116.000000,4.000000,1.000000


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574 entries, 0 to 2573
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        2574 non-null   int64 
 1   ethnicity  2574 non-null   int64 
 2   gender     2574 non-null   int64 
 3   pixels     2574 non-null   object
dtypes: int64(3), object(1)
memory usage: 80.6+ KB


Limpiamos la base de datos

In [14]:
del df['img_name']

**Primer algoritmo: Arbol de decision**

In [4]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()

In [5]:
classifier.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

Determinamos la variable X y la variable y

In [6]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# Assuming your DataFrame is named 'df' and the column is named 'pixels'
max_length = df['pixels'].str.split().str.len().max()  # Find the maximum length of pixel values

df['pixels'] = df['pixels'].apply(lambda x: np.array(list(map(float, x.split()))))
df['pixels'] = df['pixels'].apply(lambda x: np.pad(x, (0, max_length - len(x)), mode='constant'))

X = np.stack(df['pixels'].values)
y = df['gender']





División en train y test

In [7]:
# Función de partición en subconjuntos de entrenamiento y pruebas.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,     # Proporción de datos usados para el grupo de evaluación.
                                                    random_state=1234, # Semilla aleatoria para la replicabilidad.
                                                    stratify=y)

Modelo de regresión logística

In [8]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs');
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

Modelo de árbol de decisión

In [15]:
from sklearn.tree import DecisionTreeClassifier



dt_model = DecisionTreeClassifier(criterion='entropy')
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

Modelo SVC

In [16]:
from sklearn.svm import LinearSVC
linear = LinearSVC(max_iter=5000)
linear.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=5000)

In [17]:
print(f"Error en entrenamiento:\t{1-linear.score(X_train, y_train):.4f}")
print(f"Error en prueba:\t{1-linear.score(X_test, y_test):.4f}")

Error en entrenamiento:	0.0067
Error en prueba:	0.3816


Score general con los primeros hiperparámetros

In [18]:
print("Score DT", dt_model.score(X, y))
print("Score LR", lr_model.score(X, y))
print("Score SVC", linear.score(X, y))

Score DT 0.8714063714063714
Score LR 0.8115773115773116
Score SVC 0.8807303807303807


Procedemos a modificar los parámetros para encontrar el mejor modelo.

In [19]:
dt_model = DecisionTreeClassifier(criterion='log_loss')
dt_model.fit(X_train, y_train)
linear = LinearSVC(max_iter=1000)
linear.fit(X_train, y_train)
lr_model = LogisticRegression(multi_class='multinomial', solver='saga');
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(multi_class='multinomial', solver='saga')

In [20]:
print("Score DT", dt_model.score(X, y))
print("Score LR", lr_model.score(X, y))
print("Score SVC", linear.score(X, y))

Score DT 0.878010878010878
Score LR 0.7804972804972805
Score SVC 0.8648018648018648


In [21]:
dt_model = DecisionTreeClassifier(criterion='gini')
dt_model.fit(X_train, y_train)
linear = LinearSVC(max_iter=10000)
linear.fit(X_train, y_train)
lr_model = LogisticRegression(solver='newton-cholesky');
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression(solver='newton-cholesky')

In [22]:
print("Score DT", dt_model.score(X, y))
print("Score LR", lr_model.score(X, y))
print("Score SVC", linear.score(X, y))

Score DT 0.8725718725718725
Score LR 0.8904428904428905
Score SVC 0.8803418803418803


Ahora vamos a implementar una red convolucional para mejorar el score

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Assuming you have already defined X and y

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)

# Normalize the input data (if needed)
# X_train = (X_train - X_train.min()) / (X_train.max() - X_train.min())
# X_test = (X_test - X_train.min()) / (X_train.max() - X_train.min())

# Create a 1D CNN model
model = Sequential()

# Add the first convolutional layer
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))

# Add a max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Add more convolutional and pooling layers if desired
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

# Flatten the feature maps
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(64, activation='relu'))

# Add an output layer with the appropriate number of classes
num_classes = len(np.unique(y))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Reshape the input data to have a third dimension (1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
57/57 [==============================] - 4s 45ms/step - loss: 137.4975 - accuracy: 0.5441 - val_loss: 36.3134 - val_accuracy: 0.5912
Epoch 2/10
57/57 [==============================] - 3s 57ms/step - loss: 13.4453 - accuracy: 0.6102 - val_loss: 7.5632 - val_accuracy: 0.6378
Epoch 3/10
57/57 [==============================] - 2s 42ms/step - loss: 5.3108 - accuracy: 0.6624 - val_loss: 8.6229 - val_accuracy: 0.5408
Epoch 4/10
57/57 [==============================] - 2s 41ms/step - loss: 3.4588 - accuracy: 0.7041 - val_loss: 5.5059 - val_accuracy: 0.6417
Epoch 5/10
57/57 [==============================] - 2s 42ms/step - loss: 2.0613 - accuracy: 0.7540 - val_loss: 4.0874 - val_accuracy: 0.6688
Epoch 6/10
57/57 [==============================] - 2s 41ms/step - loss: 0.9801 - accuracy: 0.8284 - val_loss: 4.4361 - val_accuracy: 0.5912
Epoch 7/10
57/57 [==============================] - 3s 55ms/step - loss: 6.8906 - accuracy: 0.6713 - val_loss: 8.5176 - val_accuracy: 0.5744
Epoch 8/1

Aquí se puede ver que la red convolucional sí aumenta el accuracy pero todavía no logramos un score perfecto así que vamos aumentar la cantidad de epochs

In [25]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
57/57 [==============================] - 2s 42ms/step - loss: 0.2088 - accuracy: 0.9356 - val_loss: 3.6110 - val_accuracy: 0.6352
Epoch 2/20
57/57 [==============================] - 2s 41ms/step - loss: 0.1925 - accuracy: 0.9456 - val_loss: 3.0816 - val_accuracy: 0.6455
Epoch 3/20
57/57 [==============================] - 3s 59ms/step - loss: 0.0826 - accuracy: 0.9645 - val_loss: 3.0064 - val_accuracy: 0.6688
Epoch 4/20
57/57 [==============================] - 2s 40ms/step - loss: 0.2964 - accuracy: 0.9106 - val_loss: 3.1450 - val_accuracy: 0.6585
Epoch 5/20
57/57 [==============================] - 2s 41ms/step - loss: 0.0977 - accuracy: 0.9589 - val_loss: 3.1444 - val_accuracy: 0.6805
Epoch 6/20
57/57 [==============================] - 2s 42ms/step - loss: 0.0935 - accuracy: 0.9661 - val_loss: 3.6265 - val_accuracy: 0.6274
Epoch 7/20
57/57 [==============================] - 2s 41ms/step - loss: 0.2021 - accuracy: 0.9339 - val_loss: 3.1743 - val_accuracy: 0.6688
Epoch 8/20
57

Los resultados nos muestran que lleagmos al accuracy de 1 en el epoch 19 por lo que podemos determinar que nuestro idenfitier funciona a la perfección con el genero de un determinado grupo de personas


Asimismo identificamos que las herramientas que deep learning son mucho más específicias y tinenen mejor capacidad de clasificación que las de machine learning